In [3]:
# utils
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import random
import unidecode

# image
from PIL import Image, ImageFilter

# openCV
import cv2

# scikit learn
from skimage import io, util
from skimage.color import rgb2gray

# imgaug
import imgaug as ia
from imgaug import augmenters as iaa

In [19]:
# loading test image
img = cv2.imread('test.jpg')

In [175]:
# draw a randomic overlay to be merged with image
def random_overlay(shape, color=(55, 55, 55)):
    import random
    import cv2
    # get image shape
    height, width, channels = shape

    # create blank white image
    overlay = 255 * np.ones(shape=[height, width, channels], dtype=np.uint8)
    
    # randomize crappy shape and position
    ## 0 - rectangle
    ## 1 - line
    ## 2 - circle
    for i in range(4):
        # get randomic shape position
        x1, y1 = [random.choice(range(0, width)), random.choice(range(0, height))]
        x2, y2 = [random.choice(range(0, width)), random.choice(range(0, height))]
        
        # select random shape
        shape_range = range(0, 2)
        randomic_shape = random.choice(shape_range)
        if randomic_shape == 0:
            cv2.rectangle(overlay, (x1, y1), (min(x2, 30), min(y2, 30)), color, -1)
        elif randomic_shape == 1:
            random_thickness = random.choice(range(0, 400))
            cv2.line(overlay, (x1, y1), (min(x2, 100), min(y2, 100)), color, random_thickness)        
        else:
            cv2.circle(overlay, (x1, y1), (x2, y2), color, -1)
    overlay = cv2.blur(overlay, (150, 150))
    return overlay

In [178]:
def crappy_pipeline(image):
    # draw random shapes on image
    overlay = random_overlay(image.shape)
    alpha = 0.6
    new_image = cv2.addWeighted(image, alpha, overlay, 1 - alpha, 0)
    Image.fromarray(new_image).show()
    return

In [179]:
crappy_pipeline(img)